# World Bank + HDI Analysis (ETL + Profiling)
This notebook performs a complete **ETL (Extract, Transform, Load)** pipeline on World Bank and HDI datasets,
generates profiling insights, and prepares a clean dataset for downstream analysis (e.g., Power BI).

## **Pipeline Steps**
1. **Extract**: Load World Bank & HDI data
2. **Transform**: Clean, filter, calculate population, merge datasets
3. **Profile**: Generate summary statistics and visualize key distributions
4. **Load**: Save processed dataset to CSV
5. **Insights**: Correlation check and recommendations


In [2]:
import sys
import subprocess
import importlib
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from IPython.display import display, HTML

## 0. Environment Setup
Auto-install required libraries so this notebook runs cleanly on any machine.

In [3]:
def install_package(package):
    """Install package via pip if not already installed."""
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"📦 Installing {package} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

In [4]:
# Install required libraries
install_package("pandas")
install_package("matplotlib")
install_package("skimpy")
install_package("seaborn")
install_package("scipy")
install_package("ipython")
install_package("openpyxl")
install_package("xlrd")
install_package("numpy")
install_package("plotly")
install_package("plotly_express")

📦 Installing skimpy ...
📦 Installing ipython ...
📦 Installing xlrd ...
📦 Installing plotly ...
📦 Installing plotly_express ...


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from skimpy import skim
import seaborn as sns
from scipy.stats import pearsonr
from IPython.display import display, HTML
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot
import os
from datetime import datetime
import re
import warnings
warnings.filterwarnings("ignore")

## 1. Extract
Load datasets from World Bank (Excel) and HDI (CSV).

In [8]:
world_bank_path = "D:\courses\Data Science\Projects\Python\world-economy-bi-project\data\WorldBank.xlsx"
hdi_path = "D:\courses\Data Science\Projects\Python\world-economy-bi-project\data\HDI.csv"

world_bank = pd.read_excel(world_bank_path)
hdi = pd.read_csv(hdi_path)

print(f"✅ World Bank data: {world_bank.shape[0]} rows, {world_bank.shape[1]} columns")
print(f"✅ HDI data: {hdi.shape[0]} rows, {hdi.shape[1]} columns")
display(HTML("<h2>World Bank Data Sample</h2>"))
display(world_bank.head())
display(HTML("<h2>HDI Data Sample</h2>"))
display(hdi.head())

✅ World Bank data: 12449 rows, 15 columns
✅ HDI data: 206 rows, 1008 columns


,Country Name,Country Code,Region,IncomeGroup,Year,"Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)",Electric power consumption (kWh per capita),GDP (USD),GDP per capita (USD),Individuals using the Internet (% of population),"Infant mortality rate (per 1,000 live births)",Life expectancy at birth (years),Population density (people per sq. km of land area),Unemployment (% of total labor force) (modeled ILO estimate)
0,Afghanistan,AFG,South Asia,Low income,2018,NaN,NaN,NaN,1.936300e+10,520.897,NaN,47.9,NaN,56.9378,1.542
1,Afghanistan,AFG,South Asia,Low income,2017,33.211,6.575,NaN,2.019180e+10,556.302,13.50,49.5,64.130,55.5960,1.559
2,Afghanistan,AFG,South Asia,Low income,2016,33.981,6.742,NaN,1.936260e+10,547.228,11.20,51.2,63.763,54.1971,1.634
3,Afghanistan,AFG,South Asia,Low income,2015,34.809,6.929,NaN,1.990710e+10,578.466,8.26,53.1,63.377,52.7121,1.679
4,Afghanistan,AFG,South Asia,Low income,2014,35.706,7.141,NaN,2.048490e+10,613.856,7.00,55.1,62.966,51.1148,1.735


,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,AFG,Afghanistan,Low,SA,180.0,0.273,0.279,0.287,0.297,0.292,...,1.86,1.88,1.66,1.62,1.66,1.41,1.32,1.38,1.38,1.38
1,AGO,Angola,Medium,SSA,148.0,NaN,NaN,NaN,NaN,NaN,...,4.09,4.53,3.97,3.59,2.79,2.64,2.28,2.18,2.18,2.18
2,ALB,Albania,High,ECA,67.0,0.647,0.629,0.614,0.617,0.624,...,12.44,11.49,13.14,12.61,14.39,14.46,12.85,12.96,12.96,12.96
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,0.728,0.739,0.742,0.748,0.755,...,49.56,49.68,55.49,59.76,64.95,75.61,65.97,68.95,68.95,68.95


## 2. Transform
Filter for 2014, calculate population in millions, drop invalid rows, and merge HDI.

In [10]:
wb_2014 = world_bank[world_bank["Year"] == 2014].copy()

wb_2014["Population (M)"] = (
    wb_2014["GDP (USD)"] / wb_2014["GDP per capita (USD)"]
).fillna(0) / 1_000_000

# Drop rows with missing GDP values
wb_2014 = wb_2014.dropna(subset=["GDP (USD)", "GDP per capita (USD)"])

# Merge with HDI (using correct column names)
merged = pd.merge(
    wb_2014,
    hdi[["iso3", "hdi_2014"]].rename(columns={"iso3": "Country Code", "hdi_2014": "HDI"}),
    how="left",
    on="Country Code"
)

print(f"🔄 Transformed data: {merged.shape[0]} rows after merge")
display(HTML("<h2>Transformed Data Sample</h2>"))
display(merged.head())

🔄 Transformed data: 201 rows after merge


,Country Name,Country Code,Region,IncomeGroup,Year,"Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)",Electric power consumption (kWh per capita),GDP (USD),GDP per capita (USD),Individuals using the Internet (% of population),"Infant mortality rate (per 1,000 live births)",Life expectancy at birth (years),Population density (people per sq. km of land area),Unemployment (% of total labor force) (modeled ILO estimate),Population (M),HDI
0,Afghanistan,AFG,South Asia,Low income,2014,35.706,7.141,NaN,2.048490e+10,613.856,7.0,55.1,62.966,51.1148,1.735,33.370856,0.479
1,Albania,ALB,Europe & Central Asia,Upper middle income,2014,12.259,7.219,2309.37,1.322820e+10,4578.670,60.1,8.9,77.813,105.4420,17.490,2.889092,0.792
2,Algeria,DZA,Middle East & North Africa,Upper middle income,2014,25.538,4.709,1362.87,2.140000e+11,5493.060,29.5,21.8,75.878,16.3425,10.207,38.958249,0.735
3,American Samoa,ASM,East Asia & Pacific,Upper middle income,2014,17.500,4.200,NaN,6.430000e+08,11525.200,NaN,NaN,NaN,278.9550,NaN,0.055791,NaN
4,Andorra,AND,Europe & Central Asia,High income: nonOECD,2014,NaN,NaN,NaN,3.350740e+09,42300.300,95.9,3.2,NaN,168.5380,NaN,0.079213,0.871


## 3. Profile
Generate a quick profile of the cleaned dataset and plot key distributions.

In [11]:
# Display skimpy profile
skim(merged)

# Create output directory for visualizations
os.makedirs("reports/images", exist_ok=True)

numeric_cols = ["GDP (USD)", "GDP per capita (USD)", "Population (M)", "HDI"]

for col in numeric_cols:
    if col in merged.columns:
        plt.figure(figsize=(8, 4))
        merged[col].hist(bins=30)
        plt.title(f"Distribution of {col}")
        plt.xlabel(col)
        plt.ylabel("Frequency")
        plt.grid(False)
        plt.savefig(f"reports/images/{col.replace(' ', '_')}_hist.png")
        plt.close()

print("📸 Histograms saved in reports/images/")

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 201    │ │ float64     │ 12    │                                                          │
│ │ Number of columns │ 17     │ │ string      │ 4     │                                                          │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━┳━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column   ┃ NA ┃ NA %     ┃ mean    ┃ sd       ┃ p0      ┃ p25      ┃ p50     ┃ p75      ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━╇━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ Year     │  0 │        0 │    2014 │        0 │    2014 │     2014 │    2014 │     2014 │    2014 │     ▇  │  │
│ │ Birth    │  7 │ 3.482587 │   21.07 │    10.39 │     7.9 │    11.93 │   18.55 │    29.27 │   47.99 │ ▇▅▃▃▃▁ │  │
│ │ rate,    │    │ 06467661 │         │          │         │          │         │          │         │        │  │
│ │ crude    │    │        7 │         │          │         │          │         │          │         │        │  │
│ │ (per     │    │          │         │          │         │          │         │          │         │        │  │
│ │ 1,000    │    │          │         │          │         │          │         │          │         │        │  │
│ │ people)  │    │          │         │          │         │          │         │          │         │        │  │
│ │ Death    │  7 │ 3.482587 │    7.78 │    2.716 │   1.127 │    6.035 │   7.517 │     9.37 │   16.43 │ ▁▆▇▅▂▁ │  │
│ │ rate,    │    │ 06467661 │         │          │         │          │         │          │         │        │  │
│ │ crude    │    │        7 │         │          │         │          │         │          │         │        │  │
│ │ (per     │    │          │         │          │         │          │         │          │         │        │  │
│ │ 1,000    │    │          │         │          │         │          │         │          │         │        │  │
│ │ people)  │    │          │         │          │         │          │         │          │         │        │  │
│ │ Electric │ 64 │ 31.84079 │    4321 │     6010 │   39.06 │    904.4 │    2620 │     5600 │   53830 │   ▇▁   │  │
│ │ power    │    │ 60199005 │         │          │         │          │         │          │         │        │  │
│ │ consumpt │    │          │         │          │         │          │         │          │         │        │  │
│ │ ion (kWh │    │          │         │          │         │          │         │          │         │        │  │
│ │ per      │    │          │         │          │         │          │         │          │         │        │  │
│ │ capita)  │    │          │         │          │         │          │         │          │         │        │  │
│ │ GDP      │  0 │        0 │ 3908000 │ 15460000 │ 3729000 │ 60480000 │ 3134000 │ 20600000 │ 1750000 │   ▇    │  │
│ │ (USD)    │    │          │   00000 │    00000 │       0 │       00 │    0000 │     0000 │ 0000000 │        │  │
│ │ GDP per  │  0 │        0 │   17780 │    27110 │   24

📸 Histograms saved in reports/images/


## 4. Load
Save the cleaned dataset for downstream analysis (e.g., Power BI).

In [12]:
os.makedirs("data", exist_ok=True)
output_path = "data/processed_data.csv"
merged.to_csv(output_path, index=False)

print(f"💾 Processed data saved to: {output_path}")

💾 Processed data saved to: data/processed_data.csv


## 5. Insights
Let's quickly check the correlation between GDP per Capita and HDI to understand the relationship.

In [13]:
correlation = merged["GDP per capita (USD)"].corr(merged["HDI"])
print(f"📈 Correlation GDP per capita vs HDI (2014): {correlation:.2f}")

if correlation > 0.7:
    print("✅ Strong positive relationship between GDP per capita and HDI")
elif correlation > 0.4:
    print("⚠️ Moderate positive relationship")
else:
    print("🔎 Weak relationship — explore further factors")

📈 Correlation GDP per capita vs HDI (2014): 0.65
⚠️ Moderate positive relationship
